In [ ]:
# Read in favorite new artists from culled ArtistHasPotential playlist
# Find recent songs <3 years from new artists
# Add to newFaves playlist

In [ ]:
# Cull both newFaves and oldFaves into CurrentYearSongCandidates